In [1]:
%pip install -q "crewai[tools]"  langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.3/573.3 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.9 MB/s eta 0:0

In [46]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')
serper_api_key = userdata.get('serper_api_key')

# Student Career Coach

### Description

Landing your first role after graduation is never easy and can be overwhelming. Our service offers a personalized career coaching plan that utilizies AI! With the ability to take in student information, our team works in partnership with students to reccomend jobs, certifications, and even provides a staged schedule that students can follow to land their first role!


### Scenario

Our Student Career Coach is tasked with assisting students from Cohort 1 in the Menlo College MSIS program with landing their first role after the completion of the program. This particular student is scheduled to graduate in December of 2024 and has stated that they would like to embark in their first chosen career role in February of 2025. After meeting with a student and reviewing their provided resume, cover sheet, assessing their professional goals, and grad program information, this is the plan we designed to further assist them.

In [47]:
from crewai import Agent, Task, Crew
import os

In [48]:
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini-2024-07-18"
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key

# Agents 🤖

1. Career Guidance Counselor - Takes student information and searches for relevant roles that the student can apply to upon graduation

2. Career Planner - Provides scheduling on how to navigate the job market and upskill to make themselves a better job candidate


In [69]:
counselor = Agent(
    role="Career Guidance Counselor",
    goal="Work with the MSIS graduate students, identify opportunites for them to develop their skills, meet with prospective employers and talk to internal and external recruiters.",
    backstory= """You are working as the Career Guidance Couselor to assist Menlo College graduate students
    on landing their first designated career role upon completion of the MSIS program. You regularly provide resources for the students to engage in that will benefit and jumpstart their career. """,
    allow_delegation=False,
	verbose=True
)


In [70]:
career_planner = Agent(
    role="Career Planner",
    goal="Based on the amount of time the student has before they finish the program, develop a plan for them to complete certificates, create or review resumes, and apply for jobs.",
    backstory= """You are working as a Career Guidance Counselor. You work directly with the rest of the team and rely on their information
    to develop and schedule a plan with each student. The goal is for students to find job placement in 6 months. Develop a schedule for them to follow that considers their
    schoolwork, and make time for completing certifications reccomended by the Career Guidance Counselor. Continue
    the plan for the duration of 6 months, anything longer and the student will have to continue on their own individual plan.""",
    allow_delegation=False,
	verbose=True
)

# Tasks ✅

1. The task of the Career Guidance Counselor is to provide names of job roles relvant to the student as well as analyze the market for courses and certificates the can aquire to make themselves more appealing


2. Provide a tailored plan for the student to follow that coordinates the progression of them landing their first role

In [71]:
from crewai_tools import ScrapeWebsiteTool
from crewai_tools import SerperDevTool
from crewai_tools import DirectoryReadTool, \
                         FileReadTool


search_tool = SerperDevTool()
docs_scrape_tool = ScrapeWebsiteTool()
directory_read_tool = DirectoryReadTool()

In [82]:
market_research = Task(
    description=(
        "You are tasked with helping a student improve their job prospects.\n\n"
        "1. Based on {student_information}, provide relevant career insights.\n"
        "2. Use the `search_tool` to search the internet for job listings that align with the student's background. "
        "Be sure to include a proper search query like: "
        "`{'search_query': 'based on education and experience'}` when using the search tool.\n"
        "3. After reviewing jobs, identify gaps in experience or education and recommend online certifications to boost qualifications.\n"
    ),
    expected_output=(
        "• List of relevant job roles the student may qualify for.\n"
        "• Online certificate/course recommendations to close experience/education gaps.\n"
        "• Updated salary range estimates based on certification gains."
    ),
    tools=[search_tool, directory_read_tool],
    agent=counselor,
)


In [83]:
make_schedule = Task(
    description=(
        "1. Based on the research identified by the Career Guidance Conselor, "
            "develop a plan to complete their job search by February."
        "2. Tailor plan around school work and divide it by week. Do not make a day by day plan.\n"

    ),
    expected_output="""A 6 month plan divided into time formated stages. With each stage a task should be completed
     that progresses the student to landing their first role. Use information from the Career Guidance Conselor.""",
    agent= career_planner,
)


# Say Hello to the CREW 👊

In [84]:
crew = Crew(
    agents=[counselor, career_planner],
    tasks=[market_research, make_schedule],
    verbose=True
)

# Welcome to the Show ❗

In [85]:
result = crew.kickoff(inputs={"student_information": """EDUCATION :
Menlo College- Master of Science in Information Systems (Specialization in Data Analytics), Bachelor of Science in Business: Finance

PROFESSIONAL EXPERIENCE  :
- JP Morgan Chase Bank – Relationship Banker
Duties: Managed a 30-million-dollar portfolio of consumer and business relationships, Analyzed client financial data to discover needs and tailor customer experience, Worked with a team to close 4 investment transactions each with a minimum value of $100,000

Bloomberg L.P. - Analytics and Sales Intern
Duties: Provided strategic recommendations for data collection based on investor workflow, Contributed to processes and practices for data usage across the organization, Managed multiple projects and presented work to executive management team on a weekly basis

SKILLS:
Data Analysis and Algorithm Design: Python (Pandas, NumPy, Scikit-learn), SQL (MySQL), Microsoft Excel

Data Visualization: Tableau, Python (Seaborn, Matplotlib), Microsoft Excel


PROJECTS:

UFC Fight Predictions
Displayed: Developed and implemented Machine Learning models (Logistic Regression, Random Forest, and Multilayer Perceptron) to accurately predict fight outcomes, Cleaned, analyzed, and processed over a decade's worth of data to effectively train and test predictive models

Ad Comparison										 Displayed: Evaluated the effectiveness of various advertisements using pivot tables and groupby functions for detailed analysis, Determined ad performance by calculating click-through rates, identifying the most effective campaigns

Analyzing Sales Performance
Displayed: Analyzed sales performance of Japanese manga series, identifying the best-selling series and top publisher, Utilized Pandas and Seaborn to discern trends, revealing relative success within dataset subsets"""})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 35439663-6dc0-4c5f-a450-fe50758fb76b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 3ac7d16f-1b4d-411a-8168-4426cda34ff5
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 3ac7d16f-1b4d-411a-8168-4426cda34ff5
       Status: Executing Task...
    └── 🤖 Agent: Career Guidance Counselor
            Status: In Progress

# Agent: Career Guidance Counselor
## Task: You are tasked with helping a student improve their job prospects.

1. Based on EDUCATION :
Menlo College- Master of Science in Information Systems (Specialization in Data Analytics), Bachelor of Science in Business: Finance

PROFESSIONAL EXPERIENCE  :
- JP Morgan Chase Bank – Relationship Banker
Duties: Managed a 30-million-dollar portfolio of consumer and business relationships, Analyzed client financial data to discover needs and tailor customer experience, Worked with a team to close 4 investment transactions each with a minimum value of $100,000

Bloomberg L.P. - Analytics and Sales Intern
Duties: Provided strategic recommendations for data collection based on investor workflow, Contributed to processes and practices for data usage across the organization, Managed multiple projects and presented work to executive management team on a weekly basis

SKILLS:
Data Analysis and Algorithm Design: Python (Pandas, NumPy, Scikit-learn), SQL (MySQ

🤖 Agent: Career Guidance Counselor
    Status: In Progress

...

# Agent: Career Planner
## Task: 1. Based on the research identified by the Career Guidance Conselor, develop a plan to complete their job search by February.2. Tailor plan around school work and divide it by week. Do not make a day by day plan.



# Agent: Career Planner
## Final Answer: 
### 6-Month Career Development and Job Search Plan

#### Stage 1: Preparation and Research (Weeks 1-4)
**Objectives:**
- Understand the job market and identify specific roles of interest.
- Begin working on foundational skills for data analytics.

**Tasks:**
- **Week 1:** Research the Data Analyst and Data Scientist roles (e.g., job descriptions, required skills) across the provided companies. Create a summary of at least 3 target roles that align with your skills.
- **Week 2:** Start the **Google Data Analytics Professional Certificate** on Coursera. Aim to complete the first half of the course by the end of this week.
- **Week 3:** Continue and complete the **Google Data Analytics Professional Cert

🚀 Crew: crew
├── 📋 Task: 3ac7d16f-1b4d-411a-8168-4426cda34ff5
│      Assigned to: Career Guidance Counselor
│      Status: ✅ Completed
│   └── 🤖 Agent: Career Guidance Counselor
│           Status: ✅ Completed
└── 📋 Task: 9585a2c2-2d42-4edb-9be3-dce9b029a91f
       Status: Executing Task...
    └── 🤖 Agent: Career Planner
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 3ac7d16f-1b4d-411a-8168-4426cda34ff5
│      Assigned to: Career Guidance Counselor
│      Status: ✅ Completed
│   └── 🤖 Agent: Career Guidance Counselor
│           Status: ✅ Completed
└── 📋 Task: 9585a2c2-2d42-4edb-9be3-dce9b029a91f
       Assigned to: Career Planner
       Status: ✅ Completed
    └── 🤖 Agent: Career Planner
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9585a2c2-2d42-4edb-9be3-dce9b029a91f                                                                     │
│  Agent: Career Planner                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 35439663-6dc0-4c5f-a450-fe50758fb76b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯